# BayesianSearchCV

1. Apa itu hyperparameter tuning?
Hyperparameter tuning adalah proses mencari kombinasi parameter terbaik untuk model machine learning. Parameter ini ditentukan SEBELUM training (seperti jumlah tree di Random Forest, learning rate, dll) dan mempengaruhi performa model.
2. Jelaskan metode BayesSearchCV!
BayesSearchCV adalah metode hyperparameter tuning yang menggunakan Bayesian Optimization. Berbeda dengan Grid Search yang mencoba semua kombinasi, atau Random Search yang mencoba secara acak, BayesSearchCV menggunakan pendekatan probabilistik untuk memilih kombinasi yang paling promising berdasarkan evaluasi sebelumnya.
3. Bagaimana cara kerja BayesSearchCV?
-Membangun model surrogate (biasanya Gaussian Process) yang memperkirakan hubungan antara hyperparameter dan performa model

-Menggunakan acquisition function untuk memilih kombinasi hyperparameter berikutnya yang paling mungkin meningkatkan performa

-Secara iteratif memperbarui model surrogate berdasarkan hasil evaluasi sebenarnya

-Fokus mengeksplorasi area yang berpotensi memberikan hasil terbaik

4. Apa kelebihan BayesSearchCV dibandingkan metode hyperparameter tuning lainnya?

-Lebih efisien: Membutuhkan lebih sedikit iterasi untuk menemukan parameter optimal

-Lebih cerdas: Belajar dari iterasi sebelumnya, tidak seperti Random Search

-Cocok untuk ruang parameter besar: Tidak perlu mengeksplorasi semua kombinasi seperti Grid Search

-Konvergen lebih cepat: Biasanya menemukan solusi baik dalam iterasi yang lebih sedikit

-Menangani berbagai tipe parameter: Bisa menangani continuous, integer, dan categorical parameters



In [8]:
# import library
!pip install scikit-optimize
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [9]:
# Load Dataset
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=0)

print(f"Data train: {X_train.shape}")
print(f"Data test: {X_test.shape}")


Data train: (112, 4)
Data test: (38, 4)


In [12]:
# Definisikan Model dan parameter yang akan dioptimasi (Ruang Hyperparameter)
model = RandomForestClassifier(random_state=42)
search_spaces = {
    'n_estimators': Integer(100, 500),
    'max_depth': Integer(3, 15),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 5),
    'max_features': Categorical(['sqrt', 'log2'])
}


# Fungsi Optimisasi Menggunakan BayesianSearchCV

In [15]:
# Inisialisasi BayesSearchCV
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=50,                    # Jumlah iterasi optimisasi
    cv=5,                         # 5-fold cross validation
    n_jobs=-1,                    # Gunakan semua core processor
    random_state=42,
    verbose=1
)


In [16]:
# Jalankan optimisasi
print("Memulai optimisasi Bayesian Search...")
bayes_search.fit(X_train, y_train)
print("Optimisasi selesai!")

print("\n=== HASIL OPTIMISASI ===")
print("Parameter terbaik:", bayes_search.best_params_)
print(f"Skor cross-validation terbaik: {bayes_search.best_score_:.4f}")

Memulai optimisasi Bayesian Search...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds fo

In [17]:

# Evaluasi Model
print("\n=== EVALUASI MODEL ===")

# Prediksi dengan model terbaik
y_pred = bayes_search.predict(X_test)

# Menghitung metrik evaluasi
accuracy = accuracy_score(y_test, y_pred)

print(f"Akurasi pada data test: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Memandingkan dengan model default (tanpa tuning)
print("\n=== PERBANDINGAN DENGAN MODEL DEFAULT ===")
default_model = RandomForestClassifier(random_state=42)
default_model.fit(X_train, y_train)
y_pred_default = default_model.predict(X_test)
accuracy_default = accuracy_score(y_test, y_pred_default)

print(f"Akurasi model default: {accuracy_default:.4f}")
print(f"Akurasi model setelah tuning: {accuracy:.4f}")
print(f"Peningkatan akurasi: {accuracy - accuracy_default:.4f}")


=== EVALUASI MODEL ===
Akurasi pada data test: 0.9737

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38


=== PERBANDINGAN DENGAN MODEL DEFAULT ===
Akurasi model default: 0.9737
Akurasi model setelah tuning: 0.9737
Peningkatan akurasi: 0.0000
